In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Block(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self,in_channels,out_channels,stride = 1):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels,in_channels,kernel_size=3,
                               stride=stride,padding=1,groups=in_channels,bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv2 = nn.Conv2d(in_channels,out_channels,kernel_size=1,
                               stride=1,padding=0,bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out

In [8]:
class MobileNet(nn.Module):
    # (128,2) means conv channels=128, conv stride=2,
    # by default conv stride=1
    cfg = [64, (128,2), 128, (256,2), 256, (512,2),
           512, 512, 512, 512, 512, (1024,2), 1024]

    def __init__(self,num_classes = 10):
        super(MobileNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3,
         stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_channels=32)
        self.linear = nn.Linear(1024, num_classes)
        self.AdaptPool = nn.AdaptiveAvgPool2d((1,1))

    def _make_layers(self,in_channels):
        layers = []
        for x in self.cfg:
            out_channels = x if isinstance(x,int) else x[0]
            stride = 1 if isinstance(x,int) else x[1]
            layers.append(Block(in_channels,out_channels,stride))
            in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        print(out.shape)
        out = self.AdaptPool(out)
        print(out.shape)
        out = out.view(out.size(0), -1)
        print(out.shape)
        out = self.linear(out)
        print(out.shape)
        return out

In [9]:
net = MobileNet()
x = torch.randn(1,3,224,224)

out = net(x)

torch.Size([1, 1024, 14, 14])
torch.Size([1, 1024, 1, 1])
torch.Size([1, 1024])
torch.Size([1, 10])
